<h1><center>Back Propagation in Neural Networks</center></h1>
<h3><center>Reading time: 20 minutes</center></h3>
<h4><center>Author: Nikolas Achatz</center></h4>

In this notebook we will learn how to finish training a neural network using back propagation. We will take our forward implementation from the previous notebook and add in back propagation. Firstly, we will go through the theory and mathematics to allow for us to back propagate. Secondly, we will code the entire process of back propagation by adding to our current setup and then we will train on a basic XOR.

![Example of a neural network](./Images/neuralnetwork.png)

Here is the neural network we have been working with. We have two inputs going into 1 hidden layer with 2 neurons and outputting to one neuron. The steps of forward propagation simply just make a prediction by calculating the output through layers of neurons. However, at the start if we recall we randomly initialize our weights and biases. Obviously, we can expect the output of this neural network to be nonsense, but using back propagation we can update these and take a step in the direction of better predictions. Let's use the following dataset as a running example for this notebook: 

<style type="text/css">
.tg  {border-collapse:collapse;border-color:#93a1a1;border-spacing:0;}
.tg td{background-color:#fdf6e3;border-color:#93a1a1;border-style:solid;border-width:1px;color:#002b36;
  font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#657b83;border-color:#93a1a1;border-style:solid;border-width:1px;color:#fdf6e3;
  font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-qr91{border-color:inherit;font-family:"Arial Black", Gadget, sans-serif !important;;font-size:22px;font-weight:bold;
  text-align:left;vertical-align:top}
.tg .tg-4arq{font-family:"Arial Black", Gadget, sans-serif !important;;font-size:22px;font-weight:bold;text-align:left;
  vertical-align:top}
.tg .tg-em9h{font-family:"Arial Black", Gadget, sans-serif !important;;font-size:22px;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-qr91">A</th>
    <th class="tg-4arq">B<br></th>
    <th class="tg-4arq">Y</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-em9h">0</td>
    <td class="tg-em9h">0<br></td>
    <td class="tg-em9h">0<br></td>
  </tr>
  <tr>
    <td class="tg-em9h">0</td>
    <td class="tg-em9h">1</td>
    <td class="tg-em9h">1</td>
  </tr>
  <tr>
    <td class="tg-em9h">1</td>
    <td class="tg-em9h">0</td>
    <td class="tg-em9h">1</td>
  </tr>
  <tr>
    <td class="tg-em9h">1</td>
    <td class="tg-em9h">1</td>
    <td class="tg-em9h">0</td>
  </tr>
</tbody>
</table>

If this looks familiar, it's just a simple XOR gate. The idea is once we are done here our model will predict the correct values in the Y column based off of the A,B inputs. Let's first show that this does not work as is. This code is imported from our first notebook and we will run it with this dataset so we can compare before and after.

In [1]:
import numpy as np


# Each layer in our neural network
class NeuralLayer:
    # Randomly initialize weights and biases based off of layer size
    def __init__(self, input_neurons, output_neurons):
        self.weights = np.random.randn(input_neurons, output_neurons)
        self.bias = np.zeros((1,output_neurons))

    # Two different activations, sigmoid by default
    def sigmoid(self, neurons):
        return 1.0/(1.0 + np.exp(-neurons))
    
    def relu(self, neurons):
        return neuron * (neurons > 0)

    # Forward pass
    def forward(self, input, activation):
        if activation == 'sigmoid':
            return self.sigmoid(input @ self.weights + self.bias)
        else:
            return self.relu(input @ self.weights + self.bias)


# Our neural net
class NeuralNetwork:
    
    # Dynamically create all layers 
    def __init__(self, input_neurons, hidden_neurons, layer_count, output_neurons = 1):
        
        # Used to ensure input neurons match inputted data
        self.neuron_safety = input_neurons
        
        # Assert we have a input and output layer at the least
        assert layer_count >= 2 and output_neurons >= 1
        
        # Input layer
        self.layers = [NeuralLayer(input_neurons, hidden_neurons)]
                
        # Hidden Layers
        for i in range(layer_count - 2):
            self.layers.append(NeuralLayer(hidden_neurons, hidden_neurons))
            
        # Output layer
        self.layers.append(NeuralLayer(hidden_neurons, output_neurons))
    
    # Forward pass for each layer
    def forward(self, inp, activation = 'sigmoid'):
        
        assert inp.shape[0] == self.neuron_safety
        
        
        for layer in self.layers:
            inp = layer.forward(inp, activation)
            
        return inp 

In [2]:
# Create a neural network with 2 inputs, 6 hidden neurons in each layer, and 5 layers 
net = NeuralNetwork(2,6,2)

# Input data (A,B)
X = np.array(([0,0],[0,1],[1,0],[1,1]))

# Expected output data 
Y = np.array([0,1,1,0])

for idx, prediction in enumerate(X):
    prediction = net.forward(prediction)
    print("Model predicted: {} | actual value: {} ".format(prediction, Y[idx]))

Model predicted: [[0.34648805]] | actual value: 0 
Model predicted: [[0.18438426]] | actual value: 1 
Model predicted: [[0.34095914]] | actual value: 1 
Model predicted: [[0.20900606]] | actual value: 0 


<br/><br/><br/><br/><h2><center>Measuring Loss</center></h2>

Clearly, we see that our model is predicting junk. This makes perfect sense since the weights of our neural network are randomly initialized. Let's start to figure out how to adjust the weights so we can train our neural network.


Firstly, we need to figure out a way to quantify our loss on our prediction. To do this we need a loss function. A loss function is simply just a way of measuring loss by looking at a datapoints true and predicted value. Let's take mean squared error as our loss function. MSE is a very common loss function for regression, we are going to use it in this derivation of back propagation due to it's simplicity, however when we implement we will be using Binary Cross Entropy for binary predictions (1 or 0). 

$$L = MSE = \frac{1}{n}\sum(Y^{predicted}_i - Y^{true}_i)^2$$


Simply, this means we take the average of all of our predicted values loss squared. For example, above we predicted 0.588 for [0,0]. The actual value we want is 0, thus we take the loss to be $(0.588 - 0)^2$ for this point. Now we can take the sum and average of all the other output neurons if we want to finish calculating our loss for the general case. In our scenario, we are using one output neuron so our measurement of error will simply be $$(Y^{predicted}_i - Y^{true}_i)^2$$.

<br/><br/><br/><br/><h2><center>Gradient Descent - Minimizing the loss</center></h2>

Gradient descent seeks to minimize this loss function or in other words, the error. This is done by descending down the cost function.

![Example of a neural network](./Images/gradDescent.png)

Looking at this example we can see that we slowly take steps to the minimum of the loss functions slope. In other words, this means we need to look at the derivative (in the 1D case) of the cost function - normally this is refered to as the gradient. The problem we run into is that we only have the predictions and the labels in our loss functions derivative. Moreso, we want to adjust our predictions, but what actually makes our predictions? If we recall, the predictions are calculated through forward propagation using weights and biases. Therefore, in back propagation and gradient descent we seek to update these weights and biases to minimize the loss function. Clearly, the derivative of the loss function doesn't do that by itself. However, this is the first step of our gradient descent algorithm, we need to take the derivative (gradient) of the loss function with respect to our predictions.

$$\frac{\partial L}{\partial Y^{p}} = 2*(Y^{predicted} - Y^{true})$$


Hopefully it's clear why we are using the MSE for the loss function here as this partial derivative is trivial (or should be).


Next we should look at how changes in weights change our predictions. Moreso, how does changing weights change our activations or outputs? Well similarly we can take the partial derivative with respect to weights of our activation function. If we remember, we used sigmoid as our activation function.
<br/><br/>

<center><h2>$Sigmoid = \frac{1}{1+e^{-w}}$</h2></center>

The partial derivative of this is 
$$ \frac{1}{1+e^{-w}}*(1-\frac{1}{1+e^{-w}})$$

<br/><br/><br/><br/><h2><center>Chain Rule</center></h2>

This is a good point in time to take a step back and remind ourselves of the chain rule of differentiation. This is simply the idea that the derivation of a function embedded with another function is just the derivative of the two functions independently multiplied. 

$$ \frac{\partial}{\partial x}f(g(x)) =  g^`(x)*f^`(x)$$

Why is this important to us? Well, to do back propagation we are continuously going to be taking the partial derivative for each layer. This means when we are calculating our gradients we can simply take the gradient for the next layer and simply multiply it to the previous layers. EX: The first layer we do back propagation on may look like: 

$$ \frac{\partial}{\partial x}f(g(x))$$

The next layer in the sequence will simply be:

$$ \frac{\partial}{\partial x}f(g(h(x)))$$

Making it very easy for us to continuously calculate our gradients for each layer we back propagate through!



<h2><center>Gradient Descent - Minimizing the loss w/ Chain Rule</center></h2>

Now that we understand the chain rule and our derivatives (gradients) of the loss function and the activation function we can begin to understand how to minimize the loss function. Well again the only thing that needs to change here is the weights and the bias for each neurons. Therefore, we need to find the rate of change of the loss function with respect to the weights and biases of our layers.

$$\frac{\partial L}{\partial w} = ?$$

<br/><br/>

Let's simplify this, since this can get very messy. Let's look at the simplified case:

![Example of a neural network](./Images/backprop1.png)



Let's look at the relationship between our error and our second set of weights. The W2 is an input to our preactivation which is in input to our activation which is then an input to our loss function. This is how we can use the chain rule derived above as these are embedded functions. Therefore we know that the loss with respect to weights is just the product of these 3 partial derivatives:


<br/><br/>

<center><b>
a: activation,
p: preactivation,
L: loss,
w2: second set of weights, and
w1: first set of weights
    </b>
</center>




$$\frac{\partial L}{\partial w2} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial p}\frac{\partial p}{\partial w}$$

<br/><br/>

Now what exactly are these derivatives? $\frac{\partial L}{\partial a}$ is simply the gradient of our loss function with respect to our predictions - shown above.


$\frac{\partial a}{\partial p}$ is just the gradient of our activation function, also shown above.

Lastly, $\frac{\partial p}{\partial w}$ is just the gradient of our preactivation with respect to weights, which clear is just the inputs since preactivation is simply a weighted sum of inputs. $i*w + b$ => $i$

![Example of a neural network](./Images/backprop2.png)


Perfect, we now have related our cost function to our weights for the last layer and can start to actually update them. Before we learn how to use this to update our weights for the last layer, what about other layers? To continue backpropagation we simply keep modeling these as inputs to other functions. Instead of taking the partial derivative of the inputs with respect to the weights we will just continue to the previous layers activation.

![Example of a neural network](./Images/backprop3.png)


Therefore, we can model the relationship for W1 as: 
$$\frac{\partial L}{\partial w1} = \frac{\partial L}{\partial a^{-1}}\frac{\partial a^{-1}}{\partial p^{-1}}\frac{\partial p^{-1}}{\partial a}\frac{\partial a}{\partial p}\frac{\partial p}{\partial w1}$$

Where the -1 super script refers to the previous layer.
<br/><br/>
<br/><br/>
<br/><br/>

<h2><center>Gradient Descent - Learning Rate</center></h2>

Next we need to define learning rate. This is simply the adjustment of magnitude of our error. Meaning this determines how big of a step we take towards the minimum of our cost function. It's important this is small so we don't overshoot or cause what's called a "exploding" gradient. This is generally set between 0.00001 and 0.1. We utilize this by multipling our loss function with respect to the weights with this learning rate.

<br/><br/><br/><br/>

<h2><center>Gradient Descent - Adjustment</center></h2>

Finally, we need to actually adjust the weights for each layer in the neural network. The first hidden layer we have already derived

$$\frac{\partial L}{\partial w2} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial p}\frac{\partial p}{\partial w}$$
More importantly, we have to include the learning rate

$$0.01 * \frac{\partial L}{\partial a}\frac{\partial a}{\partial p}\frac{\partial p}{\partial w}$$


Finally, for the actual adjustment we will take the weights of the layer we are in and take away this product to yield our new weights.

$$Updated = current - 0.01 * \frac{\partial L}{\partial a}\frac{\partial a}{\partial p}\frac{\partial p}{\partial w}$$


Voila! We derived backpropagation for one hidden layer. For our second layer and so on: 

$$Updated = current - 0.01 * \frac{\partial L}{\partial a^{-1}}\frac{\partial a^{-1}}{\partial p^{-1}}\frac{\partial p^{-1}}{\partial a}\frac{\partial a}{\partial p}\frac{\partial p}{\partial w1}$$

<br/><br/><br/><br/>


<h2><center>Gradient Descent - Simplified</center></h2>


This can be complicated, but let's simplify it. All we need to do is start by taking the gradient of our loss function at the end of our forward pass (whatever it may be), in this case MSE:

$$\frac{\partial L}{\partial a} = 2*(preds - labels)$$

Next all we need is the gradient of our activation (whatever it may be), in this case sigmoid:

$$\frac{\partial a}{\partial p} = \frac{1}{1+e^{-w}}*(1-\frac{1}{1+e^{-w}})$$


Truly, this is all we need to do our back propagation now. We will set our initial gradient variable to the MSE gradient and multiply it against the activation gradient. Now for each layer we will take that product and multiply against the inputs and the weights independently and save the outcomes. The product of our gradients with the inputs of this layer will be the adjustment we will make to this layer, the product with the current weights will be sent to the next layer as per our equation above.

<br/><br/><br/><br/>

<h2><center>Code</center></h2>

As we saw above, the model currently predicts junk. This mean we need to implement back propagation, this starts with a loss function. For the example above we used MSE for simplicity, however this isn't the greatest loss function for binary prediction. A good choice of loss function is binary cross entropy, but don't worry the same methods apply. We will simply take binary cross entropy gradient and use that in replacement of MSE gradient. For sanity, we will implement both.

In [3]:
def meanSquaredError(self, preds, labels):
    return  (self.preds - self.labels)**2

def meanSquaredErrorGrad(self):
    return 2 * (self.preds - self.labels)

def binaryCrossError(self, preds, labels):
    return -labels*np.log(preds) + (1 - labels)*np.log(1-preds)

def binaryCrossErrorGrad(self):
    return -1* ((self.labels / self.preds) - (1-self.labels) / (1-self.preds))

<br/><br/>
Remember since we only have 1 output neuron we are not taking the average as you would in traditional MSE! This means we can simply implement this as shown up. 

We used sigmoid and as such we need the derivative of our activation function implemented as well.

In [4]:
def sigmoidBackward(self, grad):
    return grad * self.act * (1 - self.act)

<br/><br/>Next we need a function that takes step of gradient descent for us using our learning rate. Remember this is just current weights minus the old weights deducted by the partial derivatives multiplied to the learning rate

In [5]:
def step(self, step_size):
    self.weights -= step_size*self.grad_weights
    self.bias -= step_size*self.grad_bias

<br/><br/>Finally, we need to implement back propagation. We will do this per layer so it will go in our Neural Layer class. Let's think about this, first we need the gradient of our loss function, then we multiply it to the gradient activation of our layer. Finally, we multiply against the inputs of that layer. This yields the adjustment we need to make to this layer. However, we can't end the function at that point we need to now be able to backpropagate to the next layer. The next part of this derivation is simply taking the value of the gradient of the loss function multiplied to the gradient of the activation and multiply it to the weights of this layer and send it back. From there was can rinse and repeat to update weights for the entire network.

In [6]:
# backward pass for this layer
def backward(self, grad):
    # Take the current gradient and multiply against the layers gradient activation
    grad = self.sigmoidBackward(np.atleast_2d(grad))

    # The adjustment is the gradient multiplied to that layers inputs
    self.grad_weights = np.matmul(self.input.T, grad)
    
    # The bias is simply a sum of gradients
    self.grad_bias = grad.sum(axis=0, keepdims=True)
    
    # Return the partial derivative with respect to the preactivation
    return grad @ self.weights.T

<br/><br/>Finally, lets put these functions in our previously developed code. Error will be evaluated at the end so it belongs in the NeuralNetwork class. All of the backward propagation steps will be in the NeuralLayer as we will compute at each layer independently. More so, we need a function that will run each layers back propagation which is simply called the backward function on each layer from back to front. After that we need each layer to take a step so we can simply implement a step function which does the same.

In [7]:
import numpy as np


# Each layer in our neural network
class NeuralLayer:
    def __init__(self, input_neurons, output_neurons):
        self.weights = np.random.randn(input_neurons, output_neurons)* np.sqrt(2. / input_neurons)
        self.bias = np.ones((1,output_neurons)) * 0.5

    def sigmoid(self, neurons):
        self.act = 1.0/(1.0 + np.exp(-neurons))
        return self.act
    
    # ADDED - activation gradient
    def sigmoidBackward(self, grad):
        return grad * self.act * (1 - self.act)

    def forward(self, input):
        self.input = np.atleast_2d(input)
        return self.sigmoid(input @ self.weights + self.bias)
        
    # ADDED - backward pass for this layer
    def backward(self, grad):
        
        grad = self.sigmoidBackward(np.atleast_2d(grad))
            
        self.grad_weights = np.matmul(self.input.T, grad)
        self.grad_bias = grad.sum(axis=0, keepdims=True)
        return grad @ self.weights.T
    
    # ADDED - adjust weights
    def step(self, step_size):
        self.weights -= step_size*self.grad_weights
        self.bias -= step_size*self.grad_bias

# Our neural net
class NeuralNetwork:
    
    # Dynamically create all layers 
    def __init__(self, input_neurons, hidden_neurons, layer_count, output_neurons = 1):
                
        # Used to ensure input neurons match inputted data
        self.neuron_safety = input_neurons
        assert layer_count >= 2 and output_neurons >= 1
        
        # Input layer
        self.layers = [NeuralLayer(input_neurons, hidden_neurons)]
                
        # Hidden Layers
        for i in range(layer_count - 2):
            self.layers.append(NeuralLayer(hidden_neurons, hidden_neurons))
            
        # Output layer
        self.layers.append(NeuralLayer(hidden_neurons, output_neurons))
    
    def forward(self, inp):
        
        assert inp.shape[0] == self.neuron_safety
        
        for layer in self.layers:
            inp = layer.forward(inp)
            
        return inp 
    
    # ADDED - for each layer in the neural net back propagate
    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
            
    # ADDED - when done adjust weights for each layer
    def step(self, step_size = 0.01):
        for layer in self.layers:
            layer.step(step_size)
    
    # ADDED - MSE
    def meanSquaredError(self, preds, labels):
        self.labels = labels
        self.preds = preds
        return  (self.preds - self.labels)**2
    
    # ADDED - MSE gradient  
    def meanSquaredErrorGrad(self):
        return 2 * (self.preds - self.labels)
        
    # ADDED - Binary cross entropy
    def Error(self, preds, labels):
        self.preds = preds
        self.labels = labels
        return -labels*np.log(preds) + (1 - labels)*np.log(1-preds)
    
    # ADDED - Binary cross entropy gradient
    def ErrorGrad(self):
        return -1* ((self.labels / self.preds) - (1-self.labels) / (1-self.preds))

<br/><br/>Great! Let's evaluate our model. Let's create a neural network with 2 inputs, 16 hidden neurons, and 4 layers. We will train on the XOR dataset above and see the difference. An epoch is simply how many times we will train each 4 data points. We will train for 50,000 epochs going through each datapoint making predictions off of it then back propagating to minimize the loss and take a step to adjust the weights for each layer.

In [8]:
# Create a neural network with 2 inputs, 2 hidden neurons in each layer, and 2 layers 
net = NeuralNetwork(2,2,2)
epochs = 50000

# Input data (A,B) for XOR
X = np.array([[0,0],[1,1], [1,0],[0,1]])

# Expected output data 
Y = np.array([[0],[0],[1],[1]])


for i in range(epochs):
    preds = []
    for idx, x in enumerate(X):
        predictions = net.forward(x)
        preds.append(float(predictions))
        loss = net.Error(predictions, Y[idx])
        loss_grad = net.ErrorGrad()
        net.backward(loss_grad)
        net.step()
        
predsRounded = [int(round(x)) for x in preds]

print("Model predicted: {}\nactual values: {} ".format(preds, Y.T))
print("Model predicted: {}\nactual values: {} ".format(predsRounded, Y.T))

Model predicted: [0.0033990771655392662, 0.5038756910441456, 0.9955799136303164, 0.4969624186014994]
actual values: [[0 0 1 1]] 
Model predicted: [0, 1, 1, 0]
actual values: [[0 0 1 1]] 


<br/><br/>Fantastic! We can clearly see that the model predicts extremely close to 0 and 1 for the correct inputs! This concludes this notebook on back propagation.